# Lab 3: Obtener información de los datos mediante metric aggregations

Cuando realizamos el ejercicio de la búsquedas facetadas vimos que Elasticsearch ofrecía dos principales tipos de agregaciones:

* **Bucket aggregations**: nos van a permitir organizar grandes datasets en gurpos dependiendo del valor de un campo.
* **Metric aggregations**: nos permiten calacular métricas como count, sum, min, max, avg... sobre campos de tipo numérico.

En este notebook vamos a ver cómo extraer información de los datos indexados utilizando las metric aggregations que nos permitiránr ealizar diversas operaciones de agregación sobre los datos de los campos de los documentos indexados que no sean de tipo text y como utilizarlas junto con las bucket aggregations.

La regla general es que las metrics aggregations realizan las operaciones sobre los datos, por ejemplo, la suma de los valores de un campo, o su media, máximos, mínimos, etc.
Las bucket aggregations nos van a servir para agrupar los datos y aplicar la operación del metric aggregation a cada uno de los buckets generados.

Además podemos filtrar los datos utilizando el contexto de búsqueda de la query.

Vamos a ver las metric aggregations más comunes y cómo funcionan a través de ejemplos. Para hacer los ejemplos, utilizaremos el índice web-logs que hemos creado y utilizado en anteriores notebooks.

Dedícale unos minutos a recordar cómo era el mapping type y los datos que tenemos en este índice. Puedes hacerlo a través de Kibana o a través de la api de Elasticsarch.

## 1. Filtrar los logs para un periodo de tiempo concreto

Para agilizar las búsquedas y para ver como funcionan las agregaciones junto el contexto de búsqueda, vamos a filtrar los logs que se hayan generado para el día 2019-01-23:

`
GET web-logs/_search
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  }
}
`

## 2. Calcular el número de bytes que se han servido por los servidores web para el periodo definido en el ejercicio anterior.

En este caso la sentencia incluirá una query y una agregacion. La agregación se aplicará sobre el resultado de la query:
* La query filtra los documentos y define el subconjunto que se va a devolver.
* La aggreción define las agregaciones a aplicar sobre el subconjunto que cumple la query.

Para realizar la suma de los bytes, utilizaremos la agregacion sum indicando el campo sobre el que hacer la suma:

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "bytes_served": {
      "sum": { "field": "http.response.body.bytes" }
    }
  }
}
`

En este caso como no se ha definido ningún bucket aggregation el metric aggregation se aplica sobre el conjunto total de los datos devueltos por la query, obteniendo el total de bytes servidos.

Al poner a 0 el valor size (size=0) Elasticsearch no devolvera ningún documento. Esto es muy útil y común cuando lo único que queremos es trabajar con los resultados de las métricas y no con los documentos que cumplen el criterio de búsqueda.


## 3. Total de bytes servidos en el preriodo en cada hora para tener mayor granularidad de los datos.

Para realizar esta consulta es necesario agrupar los documentos de forma horaria por el campo @timestamp. Esto lo podemos realizar utilizando la bucket aggregation date_histogram.

Las agregaciones se pueden anidar, de esta forma le decimos a Elasticserach el orden en que quermos que se calculen y por tanto podemos decirle que primero agrupe por hora ejecutando la agregación date_histogram y a continuación la agregación de suma:

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "hourly": {
      "date_histogram": {
        "field": "@timestamp",
        "fixed_interval": "1h"
      },
      "aggs": {
        "bytes_served": { "sum": { "field": "http.response.body.bytes" } }
      }
    }
  }
}
`

La agregación de suma se ejecutará anidada para cada bucket generado por la agregación date_histogram. El resultado nos muestra para cada bucket horario la suma de bytes de los documentos que han entrado en ese bucket.

Debería haber un total de 24 buckets ya que el periodo de búsqueda se ha definido para un día concreto y las agregaciones se aplican sobre el resultado de la búsqueda.

### Ejercicio 1: Muestra el toal de bytes servidos para cada código de respuesta por hora.

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "code": {
      "terms": {
        "field": "http.response.status_code",
        "size": 20
      },
      "aggs": {
        "hourly": {
          "date_histogram": {
            "field": "@timestamp",
            "fixed_interval": "1h"
          },
          "aggs": {
            "bytes_served": { "sum": { "field": "http.response.body.bytes" } }
          }
        }
      }
    }
  }
}
`

## 4. Incrementar el periodo a 2 días para ver la estacionlidad de los datos. 

Vamos a incrementar el valor de los datos en 2 días para ver si hay cierta estacionalidad en los datos.

Además de agrupar por hora como hacíamos en el ejercicio anterior, vamos a devolver el total de datos devueltos en el periodo y en Kilobytes (KB).

Para añadir el total de datos devueltos, añadiremos un nuevo grupo de agregacion al que ya teníamos creado. Elasticsearch no permite añadir todos los gurpos de agregación que necesitemos y podremos obtener todos los datos que necesitamos en una sóla consulta.

Para devolver el valor en Kilobytes (los documentos tienen este dato en bytes), utilizaremos el parámetro script de la agregación suma, que nos permite crear un script en painless para trabajar con los datos devueltos:

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-25T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "hourly": {
      "date_histogram": {
        "field": "@timestamp",
        "fixed_interval": "1h"
      },
      "aggs": {
        "bytes_served": { "sum": { "field": "http.response.body.bytes" } }
      }
    },
    "bytes_served": {
      "sum": { 
        "field": "http.response.body.bytes",
        "script": { "source": "_value/(1024)" }
      }
    }
  }
}
` 

### Ehercicio 2: El equipo de desarrollo quiere entender que tipo de navegador genera la mayoría del tráfico (número de peticiones). 

A la información ya devuelta añade un nuevo grupo de agregación que indique cuantas peticiones se han realizado por user agent y devuelve el top 5

Nota: No existe una metric aggregation de tipo count ya que las bucket aggregations ya nos indican el número de documentos que entran en cada bucket.

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "hourly": {
      "date_histogram": {
        "field": "@timestamp",
        "fixed_interval": "1h"
      },
      "aggs": {
        "bytes_served": { "sum": { "field": "http.response.body.bytes" } }
      }
    },
    "user_agents": {
      "terms": {
        "field": "user_agent.name",
        "order": { "_count": "desc" },
        "size": 5
      }
    },
    "bytes_served": {
      "sum": { 
        "field": "http.response.body.bytes",
        "script": { "source": "_value/(1024)" }
      }
    }
  }
}
`


## 5. El quipo de istemas quire conocer la media de trafico generado por tipo de petición para poder balancear correctamente las cargas de los servidores.

* Para calcular la media utilizaremos la agregación avg que nos permite obtener la media de los valores de un campo.
* Por otro lado agurparemos los datos utilizando una agregación terms de tipo bucket que agrupe por cada uno de los request methods de las peticiones.

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.method",
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "avg_response_data": { "avg": { "field": "http.response.body.bytes" } }
      }
    }
  }
}
`

## 6. Encontrar cual es el origen (referer) que consume más y menos datos para poder establecer un sistema de calidad de servicio priorizando los orígenes con menos volumen de datos.

* Parecido al ejercicio anterior, pero en este caso utilizaremos las metric aggregations max y min

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.referrer",
        "order": {
          "_count": "desc"
        }
      },
      "aggs": {
        "max_response_data": { "max": { "field": "http.response.body.bytes" } },
        "min_response_data": { "min": { "field": "http.response.body.bytes" } }
      }
    }
  }
}
`

## 7. Obtener todas las estadísticas del campo body.bytes por origen de las peticiones

* Si lo que queremos es obtener todas las metricas de un campo sin tener que utilizar diferentes agregaciones, podemos utilizar la metric aggregation stats.

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.referrer",
        "order": {
          "_count": "desc"
        }
      },
      "aggs": {
        "response_body_stats": { "stats": { "field": "http.response.body.bytes" } }
      }
    }
  }
}
`

## 8. Numero de peticiones diferentes que se han realizado por metodo.

* La metric aggregation cardinality calcula la cardinalidad de los valores de un campo, es decir el número de valores diferentes que tenemos en un campo entre todos los documentos indexados.

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.method",
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "request_count": { "cardinality": { "field": "url.original" } } 
      }
    }
  }
}
`

## 9. Con el fin de conocer la distribución de carga horaria de los servidores, calcular la media de peticiones horaria.

Además de las agregaciones de tipo bucket y metric, existen otro tipo de agregaciones que se denominan **pipeline aggregation'**. Estas agregaciones se utilizan para construir un pipeline de agregación donde se pueden definir diferentes agregaciones y utilizar el resultado de agregaciones anteriores en una nueva agregación.

Para poder reutilizar el resultado de una agregación, se utiliza lo que se denomina "bucket path" y para expresar este path se utiliza una sintaxis especifica: https://www.elastic.co/guide/en/elasticsearch/reference/8.4/search-aggregations-pipeline.html#buckets-path-syntax

* Para acceder al valor de una agregación anterior, utilizamos el como path el nombre de la agregación. Si estas están anidadas utilizaremos el operador '>' para concatenarlas.
* Para acceder al valor concreto usaremos el operador '.' y el nombre del valor.

Por ejemplo, en la consulta del ejercicio anterior, si queremos acceder al valor de la cardinalidad del campo "url.original" agregado por metodo de la petición que hemos calculado:

`
 "aggs": {
    "method": {
      "terms": {
        "field": "http.request.method",
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "request_count": { "cardinality": { "field": "url.original" } } 
      }
    }
  }
`

Utilizaríamos el siguiente bucket path: method>request_count.cardinality

Para trabajar con estos valores podemos utilizar un nuevo conjutno de aggregations que realizan operaciones similares a las metric aggregations, pero iterando sobre los valores del resultado de las agregaciones aneriores.
Por ejemplo, tenemos la max_bucket que te calcula el valor máximo de los ersultados agrupados en una aggregación anterior. Así por ejemplo, si quisiesemos saber cual es la cardinalidad máxima podremos crear la siguiente agregación:

`
"request_avg": {
    "max_bucket": {
        "buckets_path": "method>request_count" 
    }  
}
`

Vamos a ver como funciona:

`
GET web-logs/_search
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.method",
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "request_count": { "cardinality": { "field": "url.original" } } 
      }
    },
    "max_cardinality": {
      "max_bucket": {
        "buckets_path": "method>request_count" 
      }  
    }
  }
}
`


### Ejercicio 3: Modifica la sentencia anterior para obtener el valor mínimo.

`
GET web-logs/_search
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-23T00:00:00.000Z",
        "lt": "2019-01-24T00:00:00.000Z"
      }
    }
  },
  "aggs": {
    "method": {
      "terms": {
        "field": "http.request.method",
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "request_count": { "cardinality": { "field": "url.original" } } 
      }
    },
    "min_cardinality": {
      "min_bucket": {
        "buckets_path": "method>request_count" 
      }  
    }
  }
}
`

Bien, ahora que ya entendemos como funcionan las agregaciones de tipo pipeline, vamos con lo que se nos pide el enunciado, la media horaria de peticiones.

No tenemos un campo que contenga la hora a la que se hizo la petición para poder agregar por él, pero si tenemos el timestamp en el que se hizo la petición. Por lo que tenemos que extraer de ese campo la hora del valor del timestamp. Para realizar esta operación, vamos a utilizar los **"runtime mappings"** que nos van a permitir generar un nuevo campo a partir del valor de otros por medio de scripts en painless. Estos campos se evaluan en tiempo de ejecución, no se almacenan en el índice.

Por tanto, en nuestro caso vamos a crear un nuevo campo de tipo long que se llame date.hour_of_day que contenga sólo la hora del timestamp (0-23):

`
"runtime_mappings": {
    "date.hour_of_day": {
      "type": "long",
      "script": "emit(doc['@timestamp'].value.getHour())"
    }
}
`

De esta forma ya podemos agrupar por hora:

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-01T00:00:00.000Z",
        "lt": "2019-12-31T23:59:59.000Z"
      }
    }
  },
  "runtime_mappings": {
    "date.hour_of_day": {
      "type": "long",
      "script": "emit(doc['@timestamp'].value.getHour())"
    }
  },
  "aggs": {
    "hour": {
      "histogram": {
        "field": "date.hour_of_day",
        "interval": 1,
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "day": {        
           "date_histogram": {
            "field": "@timestamp",
            "calendar_interval": "1d"
          },
          "aggs": {
            "total" : { "sum" : { "field" : "agg.count" } }
          }
        }
      }
    }
  }
}
`

Ya tenemos por acada hora y día el número de peticiones. Sólo nos queda calcular la medía horaria para todos los días. Vamos a utilizar una agregación de tipo pipeline, para poder acceder a estos valores y calcular la meda por hora:

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-01T00:00:00.000Z",
        "lt": "2019-12-31T23:59:59.000Z"
      }
    }
  },
  "runtime_mappings": {
    "date.hour_of_day": {
      "type": "long",
      "script": "emit(doc['@timestamp'].value.getHour())"
    },
    "agg.count": {
      "type": "long",
      "script": "emit(1)"
    }
  },
  "aggs": {
    "hour": {
      "histogram": {
        "field": "date.hour_of_day",
        "interval": 1,
        "order": {
          "_key": "asc"
        }
      },
      "aggs": {
        "day": {        
           "date_histogram": {
            "field": "@timestamp",
            "calendar_interval": "1d"
          },
          "aggs": {
            "total" : { "sum" : { "field" : "agg.count" } }
          }
        },
        "request_avg": {
          "avg_bucket": {
            "buckets_path": "day.total" 
          }  
        }
      }
    }
  }
}
`

* Puesto que no existe una metric aggregation que cuente el número de documentos y no podemos crear un bucket_path que nos devuelva el número de documentos que contiene un bucket, porque no es una métrica. Lo que vamos a hacer es crear un nuevo campo con un mapping dinámico de tipo long que hemos llamado "agg.count" que contiene el valor 1. De esta manera lo podemos sumar y obtener así el número total de documentos para un bucket con la metric aggregation sum.

* Ya podemos crear la pipeline aggregation "avg_bucket" indicando en el bucket_path la ruta hasta la metrica de suma: "day.total".

### Ejercicio 4: Conocer la media de peticiones horarias por país

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-01T00:00:00.000Z",
        "lt": "2019-12-31T23:59:59.000Z"
      }
    }
  },
  "runtime_mappings": {
    "date.hour_of_day": {
      "type": "long",
      "script": "emit(doc['@timestamp'].value.getHour())"
    },
    "agg.count": {
      "type": "long",
      "script": "emit(1)"
    }
  },
  "aggs": {
    "country": {
      "terms": {
        "field": "source.geo.country_name"
      },
      "aggs": {
        "hour": {        
          "histogram": {
            "field": "date.hour_of_day",
            "interval": 1,
            "order": {
              "_key": "asc"
            }
          },
          "aggs": {
            "number_of_requests" : { "sum" : { "field" : "agg.count" } }
          }
        },
        "request_avg": {
          "avg_bucket": {
            "buckets_path": "hour.number_of_requests" 
          }  
        }
      }
    }
  }
}
`

### Ejercicio 5: Conocer la media de peticiones horarias por país y por país y ciudad

`
GET web-logs/_search?size=0
{
  "query": {
    "range": {
      "@timestamp": {
        "gte": "2019-01-01T00:00:00.000Z",
        "lt": "2019-12-31T23:59:59.000Z"
      }
    }
  },
  "runtime_mappings": {
    "date.hour_of_day": {
      "type": "long",
      "script": "emit(doc['@timestamp'].value.getHour())"
    },
    "agg.count": {
      "type": "long",
      "script": "emit(1)"
    }
  },
  "aggs": {
    "country": {
      "terms": {
        "field": "source.geo.country_name",
        "size": 200
      },
      "aggs": {
        "city":{
          "terms": {
            "field": "source.geo.city_name",
            "size": 200
          }, 
          "aggs": {
            "hour": {        
              "histogram": {
                "field": "date.hour_of_day",
                "interval": 1,
                "order": {
                  "_key": "asc"
                }
              },
              "aggs": {
                "number_of_requests" : { "sum" : { "field" : "agg.count" } }
              }
            },
            "request_avg": {
              "avg_bucket": {
                "buckets_path": "hour.number_of_requests" 
              }  
            }
          }
        },
        "country_hour": {        
          "histogram": {
            "field": "date.hour_of_day",
            "interval": 1,
            "order": {
              "_key": "asc"
            }
          },
          "aggs": {
            "number_of_requests" : { "sum" : { "field" : "agg.count" } }
          }
        },
        "request_avg": {
          "avg_bucket": {
            "buckets_path": "country_hour.number_of_requests" 
          }  
        }
      }
    }
  }
}
`